## Time Transforms

Run this notebook in Vertex Workbench. In this notebook, we will use extracts from the timestamp
as input features.

## Parts of time

Use extracted value from the timestamp

In [ ]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  EXTRACT(hour FROM dep_time) AS dep_hour
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
JOIN taxiout_by_airport USING(ORIGIN)
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False
LIMIT 5

In [57]:
%%bigquery
WITH taxiout_by_airport AS (
   SELECT 
    ORIGIN, AVG(taxi_out) AS avg_taxi_out
   FROM
    dsongcp.flights_tzcorr
   GROUP BY ORIGIN
)

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  AVG(dep_delay) OVER (origin_time_window) AS avg_dep_delay,
  taxi_out,
  avg_taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
JOIN taxiout_by_airport USING(ORIGIN)
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False
WINDOW origin_time_window AS (PARTITION BY ORIGIN
                              ORDER BY UNIX_SECONDS(dep_time) 
                              RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING)

LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.85rows/s]


,ontime,dep_delay,avg_dep_delay,taxi_out,avg_taxi_out,distance,origin,dest,is_eval_day
0,ontime,-8.0,NaN,12.0,13.313258,425.0,ABE,DTW,False
1,ontime,-7.0,-8.0,13.0,13.313258,692.0,ABE,ATL,False
2,ontime,0.0,NaN,9.0,13.313258,692.0,ABE,ATL,False
3,ontime,0.0,NaN,9.0,13.313258,425.0,ABE,DTW,True
4,ontime,7.0,0.0,8.0,13.313258,654.0,ABE,ORD,True


In [52]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_timefeatures
OPTIONS(input_label_cols=['ontime'], 
        model_type='boosted_tree_classifier',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS

WITH taxiout_by_airport AS (
   SELECT 
    ORIGIN, AVG(taxi_out) AS avg_taxi_out
   FROM
    dsongcp.flights_tzcorr
   GROUP BY ORIGIN
)

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  AVG(dep_delay) OVER origin_time_window AS avg_dep_delay,
  taxi_out,
  avg_taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
JOIN taxiout_by_airport USING(ORIGIN)
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False
WINDOW origin_time_window AS (PARTITION BY ORIGIN
                              ORDER BY UNIX_SECONDS(dep_time) 
                              RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1655.86query/s]                        


""


In [61]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_timefeatures,
                 (
WITH taxiout_by_airport AS (
   SELECT 
    ORIGIN, AVG(taxi_out) AS avg_taxi_out
   FROM
    dsongcp.flights_tzcorr
   GROUP BY ORIGIN
)
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  AVG(dep_delay) OVER origin_time_window AS avg_dep_delay,
  taxi_out,
  avg_taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
JOIN taxiout_by_airport USING(ORIGIN)
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
WINDOW origin_time_window AS (PARTITION BY ORIGIN
                              ORDER BY UNIX_SECONDS(dep_time) 
                              RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING)
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.01rows/s]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.842506,1305656,0.965773,283750,0.20406


Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.